In [12]:
import requests

ip_address = '127.0.0.1'
ip_address = '52.52.144.73'

def run_test(url, reason):
    response = requests.get(url)
    print(f'Testing: {url}')
    print(f'Reason: {reason}')
    print(f'Status Code: {response.status_code}')
    print(f'Response Content: {response.json()}')
    print('---------------------\n')

# 200 完全符合捷運站名
run_test(f'http://{ip_address}:3000/api/attractions?keyword=台北101／世貿', '完全符合捷運站名')
run_test(f'http://{ip_address}:3000/api/attractions?keyword=／世貿', '回傳空值，捷運站名不完整')

# 200 模糊比對景點名稱
run_test(f'http://{ip_address}:3000/api/attractions?keyword=北投', '模糊比對景點名稱')

# 500 錯誤-頁碼非正整數
run_test(f'http://{ip_address}:3000/api/attractions?page=-1', '錯誤-頁碼非正整數')
run_test(f'http://{ip_address}:3000/api/attractions?page=abc', '錯誤-頁碼非正整數')

# 200 根據景點編號取得景點資料
run_test(f'http://{ip_address}:3000/api/attraction/1', '根據景點編號取得景點資料')
run_test(f'http://{ip_address}:3000/api/attraction/20', '根據景點編號取得景點資料')

# 400 景點編號不正確
run_test(f'http://{ip_address}:3000/api/attraction/200', '景點編號不正確')
run_test(f'http://{ip_address}:3000/api/attraction/-1', '景點編號不正確')

# 500 錯誤-ID非整數
run_test(f'http://{ip_address}:3000/api/attraction/abc', '錯誤-ID非整數')

# 200 取得前40筆資料，並且依照提到的捷運站數目，排序後回傳捷運站名稱
run_test(f'http://{ip_address}:3000/api/mrts', '取得所有捷運站名稱列表，按照週邊景點的數量由大到小排序')

# 500 資料庫讀取錯誤
# run_test(f'http://{ip_address}:3000/api/mrts', '資料庫讀取錯誤')

Testing: http://127.0.0.1:3000/api/attractions?keyword=台北101／世貿
Reason: 完全符合捷運站名
Status Code: 200
Response Content: {'nextPage': 1, 'data': [{'id': 25, 'name': '台北101', 'category': '其\u3000\u3000他', 'description': '台北101購物中心為地上5樓，地下1樓的購物空間，23000坪，是臺灣首座國際頂級購物中心。擁有許多精品旗艦店，如BALLY、LV、Prada、Gucci、Cartier、DIOR及FENDI等，消費者可以享受到最多樣的選擇，與全球流行零時差，輕鬆擁有愉悅的購物時刻。 臺北101購物中心有歐式、日式、泰式、中式等各國風味美食餐廳，如代官山居食屋、晶湯匙泰式料理、隨意鳥地方、九如浙江美食、川滇食尚等，美食街擁有上千個座位，非常舒適的用餐環境。4樓都會廣場擁有完美的空間設計規劃，佔地500餘坪，挑高40米。採光自然，與四周的露天座椅融合為一體，散發著明亮開闊的現代氛圍。與世貿中心和ATT 4 FUN購物中心有空橋連接，近君悅飯店，交通十分方便。 ', 'address': '臺北市  信義區信義路5段7號', 'transport': '1.捷運：信義線「台北101站」4號出口或板南線「市政府站」2號出口步行約10分鐘2.公車：搭乘至市府站、世貿站、信義行政中心、君悅飯店路線之公車1、20、22、28、32(區)、33、37、38、46、202、207、226、266、277、281、282、284、288、292、537、611、612、621、625、647、650、651、665、669、902、信義、博愛、藍5、藍10、板基線3.車位： 松智路及市府路各二處停車場出入口TAIPEI 101 Mall B2-B4 共1100個車位週邊 500公尺內共15座停車場', 'mrt': '台北101／世貿', 'lat': '25.033194', 'lng': '121.564837', 'images': ['https://www.travel.taipei/d_upload_ttn/sceneadmin/pic/11000361.jpg',

In [ ]:
import json

with open('data/taipei-attractions.json', encoding='utf-8-sig') as file:
    data = json.load(file)
    data = data['result']['results']

pattern = 'https://'
ret = []
for spot in data:
    spot['file'] = [pattern + i for i in spot['file'].split(pattern) if ('.jpg' in i) | ('.png' in i) | ('.JPG' in i) | ('.PNG' in i)]
    temp_dict = {k.lower(): v for k, v in spot.items()}
    temp_dict['mrt'] = '週邊無捷運站' if not temp_dict['mrt'] else temp_dict['mrt']
    ret.append(temp_dict)

temp = {}
for i in ret[:40]:
    temp[i['mrt']] = temp.get(i['mrt'], 0)
    temp[i['mrt']] += 1


sorted_data = sorted(temp.items(), key=lambda x: x[1], reverse=True)
sorted_mrts = [item[0] for item in sorted_data]
sorted_mrts
# temp